# Topic Modeling with SVD


## What is "topic modeling"?

Suppose you're given a large text corpus containing several documents. You'd like to know the key topics that reside in the given collection of documents without reading through each document.

Topic Modeling helps you distill the information in the large text corpus into a certain number of topics. Topics are groups of words that are similar in context and are indicative of the information in the collection of documents.

The general structure of the Document-Term Matrix (DTM) for a text corpus containing M documents, and N terms in all is shown below:

<img src="./images/doc-term-matrix.png" width="250px"/>

Let's parse the matrix representation:

- D1, D2, ..., DM are the M documents.
- T1, T2, ..., TN are the N terms

To populate the Document-Term Matrix, let’s use the widely-used metric—the TF-IDF Score.

## Term-Frequency Inverse Document Frequency (TF-IDF)

[TFIDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf–idf value increases proportionally to the number of times a word appears in the document and is offset by the number of documents in the corpus that contain the word, which helps to adjust for the fact that some words appear more frequently in general. tf–idf is one of the most popular term-weighting schemes today. A survey conducted in 2015 showed that 83% of text-based recommender systems in digital libraries use tf–idf.

- **Term Frequency**: If we rank text based on TF, we would be using the weight of a term that occurs in a document and is simply proportional to the term frequency.

- **Inverse document frequency**: The specificity of a term can be quantified as an inverse function of the number of documents in which it occurs. 

The weighting of a term in a document based on TF-IDF can be expressed as:

$$ w_{ij} = TF_{ij} log(\frac{M}{df_j}) $$

where, 

- $TF_{ij}$ is the number of times the term  $T_j$ occurs in the document $D_i$.
- $df_j$ is the number of documents containing the term $T_j$

Intuition: 
- A term that occurs frequently in a particular document, and rarely across the entire corpus has a higher IDF score.
- Common words such as "the" or "of" which happens in every document will have high $df_j$ and a low overall TF-IDF $w_{ij}$ score.

## Topic Modeling with SVD

Singular Value Decomposition (SVD) on the the Document-Term Matrix D (of dimension MxN) gives the following three matrices:

$D = U . S. V^T$

Recall these 3 matrices $U$, $S$, and $V$:

- $U$ has dimensions $MxM$, $S$ is a diagonal matrix with dimensions $M×N$, and $V$ has dimensions $NxN$.  
  -- For a DTM, $U$ is also called the **document similarity matrix**. The i,j-th entry of the document similarity matrix signifies how similar document i is to document j.
  -- The right singular vector matrix $V$, which is also called the **term topic matrix**. The topics in the text reside along the rows of this matrix. 

- $U$ and $V$ are orthogonal matrices. That is, $U^⊤U=I$ and $V^⊤V=I$. This also implies that all columns of $U$ and $V$ have magnitude 1 and are mutually orthogonal.

- $S$ is a diagonal matrix. That is, all elements in $S$ are 0 unless they lie on the diagonal. In addition, the diagonal elements in $S$ are arranged from biggest to smallest.  The matrix of singular values $S$ which signify the relative importance of topics -- per their corresponding singular values.

One may think of SVD as a black box that operates on your Document-Term Matrix (DTM) and yields 3 matrices, U, S, and V. And **the topics reside along the rows of the matrix V_T**. This is illustrated in the figure below:

<img src="images/svd-topic-modeling.png" width="500px"/>

### What is Truncated SVD or k-SVD?

Suppose you have a text corpus of 150 documents. Would you prefer skimming through 150 different topics that describe the corpus, or would you be happy reading through 10 topics that can convey the content of the corpus?

Well, it's often helpful to fix a small number of topics that best convey the content of the text. And this is what motivates k-SVD.

As matrix multiplication requires a lot of computation, it's preferred to choose the k largest singular values, and the topics corresponding to them. The working of k-SVD is illustrated below.

<img src="images/svd-ktopic-modeling.png" width="700px"/>


## Text Corpus SVD

Let's apply SVD topic modeling to a set of BBC news articles

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [11]:
df = pd.read_csv('../data/bbc_text.csv')
print(df.shape)
df.head(5)

(2225, 2)


,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


Let's first vectorize the input "text" into a vector of terms per document:

In [12]:
vectorizer = TfidfVectorizer(stop_words='english',smooth_idf=True) 

# under the hood - lowercasing,removing special chars,removing stop words
DTM = vectorizer.fit_transform(df["text"]).todense()
DTM.shape

(2225, 29126)

So far, we have:

- identified the text corpus,

-  imported the necessary modules, and

- obtained the input DTM.

Next, let's proceed with using SVD to obtain topics using the TruncatedSVD class.

In [13]:
svd_modeling = TruncatedSVD(n_components=4, algorithm='randomized', n_iter=100, random_state=42)
svd_modeling.n_components

4

In [15]:
svd_modeling.fit(DTM)
components=svd_modeling.components_
vocab = vectorizer.get_feature_names()

/Users/pmui/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/pmui/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
len(vocab)

29126

In [18]:
components.shape

(4, 29126)

Let’s write a function that gets the topics for us.

In [27]:
topic_word_list = []
def get_topics(components):
    for i, comp in enumerate(components):
        terms_comp = zip(vocab,comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
        topic = ""
        for t in sorted_terms:
            topic = topic + ' ' + t[0]
        topic_word_list.append(topic)
    return topic_word_list

In [28]:
get_topics(components)

[' said mr labour people year election blair',
 ' mr labour election blair brown party tax',
 ' england game win best wales ireland cup',
 ' film best awards award oscar actor actress']

In [32]:
svd_modeling.singular_values_

array([7.26165116, 4.77370181, 4.21364456, 3.93105517])

In [36]:
# Percentage of variance explained by each of the selected components.

svd_modeling.explained_variance_ratio_

array([0.00311798, 0.00993334, 0.00815029, 0.00709584])

In [35]:
svd_modeling.explained_variance_ratio_.sum()

0.028297453580367105

In [39]:
svd_modeling.n_features_in_

29126